1. Írjunk **tárolt eljárást**, amely meghatározza azokat a kocsmákat, amelyekben legalább 2 olyan italt árulnak, mint a paraméterként megadott kocsma (@KocsmaNev varchar(30) típusú)! Ha nincs ilyen kocsma, írjunk ki hibaüzenetet, majd szúrjuk be a kocsmát az adatbázisba!

In [49]:
CREATE OR ALTER PROCEDURE KocsmakEsItalok (
    @KocsmaNev varchar(30)
)
AS
BEGIN
    DECLARE @KID INT
    DECLARE @newID INT

    SELECT @KID = KocsmaID FROM Kocsmak 
    WHERE Nev = @KocsmaNev

    IF @KID IS NULL
    BEGIN
        PRINT 'NINCS ILYEN KOCSMA'
        SELECT @newID = (SELECT COUNT(KocsmaID) FROM Kocsmak) + 1
        INSERT INTO Kocsmak (KocsmaID, Nev, Cim, NyitasDatuma) VALUES (@newID, @KocsmaNev, 'UNKNOWN', GETDATE())
        
        RETURN
    END

    SELECT k.Nev AS KocsmaNeve, COUNT(*) AS ItalSzam FROM Kocsmak k 
    INNER JOIN Arak a ON k.KocsmaID = a.KocsmaID
    WHERE a.ItalID IN (
            SELECT ItalID FROM Arak
            WHERE KocsmaID = @KID AND K.KocsmaID != @KID
        )
    GROUP BY k.Nev
    HAVING COUNT(*) >= 2
END

Commands completed successfully.

Total execution time: 00:00:00.008

In [50]:
EXEC KocsmakEsItalok @KocsmaNev = 'Sandra Tweed'

(3 rows affected)

Total execution time: 00:00:00.020

KocsmaNeve,ItalSzam
Frederico Sawley,3
Hedwiga Ledbetter,2
Orren Broadhurst,2


In [30]:
EXEC KocsmakEsItalok @KocsmaNev = 'Planetarium'

NINCS ILYEN KOCSMA

(1 row affected)

Total execution time: 00:00:00.007

In [35]:
SELECT * FROM Kocsmak
SELECT * FROM Baratok
SELECT * FROM ItalTipusok

(13 rows affected)

(16 rows affected)

(5 rows affected)

Total execution time: 00:00:00.046

KocsmaID,Nev,Cim,NyitasDatuma
1,Frederico Sawley,9 Logan Way,2014-12-06
2,Orren Broadhurst,87124 Basil Avenue,2016-12-20
3,Daisy Geeve,5265 Maple Wood Point,2010-08-26
4,Hedwiga Ledbetter,6 Packers Parkway,2012-07-07
5,Talia Henrionot,20571 Waxwing Junction,2016-12-07
6,Nilson Allflatt,954 Roth Pass,2017-09-16
7,Lewes Ashbrook,10 Clyde Gallagher Crossing,2012-10-08
8,Sandra Tweed,8 Superior Alley,2011-08-28
9,Perrine Chuney,26 Almo Road,2015-10-12
10,Sosanna Segeswoeth,4 Cody Park,2013-08-30


BaratID,Nev,Tel,Email,Cim
1,Fern Dunphy,6483578626,fdunphy0@princeton.edu,8261 Waxwing Place
2,Dinnie Hebble,1821798103,dhebble1@angelfire.com,76066 Tennessee Road
3,Prisca Dronsfield,8331573181,pdronsfield2@guardian.co.uk,5824 Brown Center
4,Dennie Billison,2911989379,dbillison3@twitter.com,9 Bay Lane
5,Elle Shortell,8449569855,eshortell4@topsy.com,291 Holy Cross Parkway
6,Gipsy Yashunin,6315387706,gyashunin5@csmonitor.com,9 Meadow Ridge Hill
7,Aarika Mashal,5523618089,amashal6@wired.com,288 Lotheville Avenue
8,Celestina Woolaston,9998149794,cwoolaston7@alibaba.gov.cn,934 Kings Center
9,Garvey Chartre,4934922638,gchartre8@alibaba.com,218 Spenser Terrace
10,Rik Turbard,8485977307,rturbard9@vistaprint.com,737 Ramsey Plaza


TipusID,TipusNev
3,konyak
4,tea
5,tonic
2,vodka
1,whiskey


2. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span> **függvényt**<span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">, melynek bemenő paraméterei: @pBaratNev, @pItaltipusNev. A függvény egy táblában térítse vissza azon kocsmá(k) nevét, címét és a kapható italok nevét és árát mely(ek)et a paraméterként megadott barát kedvel és ahol a megadott italtípusból kapható ital!</span>

In [43]:
CREATE OR ALTER FUNCTION KedvencKocsmaEsItal (
    @pBaratNev VARCHAR(30),
    @pItaltipusNev VARCHAR(30)
)
RETURNS @Visszaterit TABLE
(
    KocsmaNev VARCHAR(30),
    KocsmaCim VARCHAR(30),
    ItalNev VARCHAR(30),
    Ar INT
)
AS
BEGIN
    INSERT INTO @Visszaterit (KocsmaNev, KocsmaCim, ItalNev, Ar)
    SELECT k.Nev, k.Cim, i.Nev, a.Ar FROM Baratok b
    JOIN Kedvencek ke ON b.BaratID = ke.BaratID
    JOIN Kocsmak k ON ke.KocsmaID = k.KocsmaID
    JOIN Arak a ON k.KocsmaID = a.KocsmaID
    JOIN Italok i ON a.ItalID = i.ItalID
    JOIN ItalTipusok it ON i.TipusID = it.TipusID
    WHERE b.Nev = @pBaratNev AND it.TipusNev = @pItaltipusNev

    RETURN
END


Commands completed successfully.

Total execution time: 00:00:00.010

In [44]:
SELECT * FROM KedvencKocsmaEsItal('Elle Shortell', 'vodka');

(2 rows affected)

Total execution time: 00:00:00.027

KocsmaNev,KocsmaCim,ItalNev,Ar
Frederico Sawley,9 Logan Way,Alexander,9
Frederico Sawley,9 Logan Way,V33,6


3. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span> **DELETE triggert**<span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">, mely akkor aktiválódik, ha törlünk</span> **egy vagy több** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">sort az ItalTipusok táblából! (</span>**Figyelem!** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">A hivatkozási épség megszorítások fenntartása miatt probléma lehet, ha egy olyan italtípust szeretnék törölni, melyhez tartozik ital. Ezt a problémát orvosoljuk a trigger segítségével! Az adott italtípus mindenképp törlődjön a trigger végrehajtásának következtében!)</span>

In [45]:
CREATE OR ALTER TRIGGER italtipustorles ON ItalTipusok
INSTEAD OF
DELETE 
AS
BEGIN
    SET NOCOUNT ON

    DECLARE @Torolve TABLE (ItalID INT)
    INSERT INTO @Torolve (ItalID)
    SELECT ItalID FROM Italok
    WHERE TipusID IN (SELECT deleted.TipusID FROM deleted)

    DELETE FROM Italok
    WHERE ItalID IN (SELECT ItalID FROM @Torolve)

    DELETE FROM Arak
    WHERE ItalID IN (SELECT ItalID FROM @Torolve)

    DELETE FROM ItalTipusok
    WHERE TipusID IN (SELECT deleted.TipusID FROM deleted)

    --DROP TABLE @Torolve(ItalID)
END


Commands completed successfully.

Total execution time: 00:00:00.026

4. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span> **INSERT** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">és</span> **UPDATE** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">triggert az Arak táblára vonatkozóan, amely csak akkor engedélyezi a művelet végrehajtását, ha az árak összértéke</span> **5** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">és</span> **1000** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">között marad!</span>

In [6]:
CREATE OR ALTER TRIGGER insertEsUpdateArak ON Arak
INSTEAD OF
INSERT
AS 
BEGIN
    SET NOCOUNT ON
    DECLARE @TeljesAr INT
    SELECT @TeljesAr = SUM(Ar) FROM Arak

    IF @TeljesAr < 5 OR @TeljesAr > 1000
    BEGIN
        PRINT 'Az arak osszerteke 5 es 1000 kozott kell legyen!'
        RAISERROR ('Az arak osszerteke 5 es 1000 kozott kell legyen!', 16, 1)
        ROLLBACK TRANSACTION
    END
    ELSE
    BEGIN
        INSERT INTO Arak (KocsmaID, ItalID, Ar)
        SELECT KocsmaID, ItalID, Ar
        FROM inserted
    END
END

Commands completed successfully.

Total execution time: 00:00:00.011

In [13]:
INSERT INTO Arak (KocsmaID, ItalID, Ar) VALUES (1, 4, 500)

Az arak osszerteke 5 es 1000 kozott kell legyen!

: Msg 50000, Level 16, State 1, Procedure insertEsUpdateArak, Line 13
Az arak osszerteke 5 es 1000 kozott kell legyen!

: Msg 3609, Level 16, State 1, Line 1
The transaction ended in the trigger. The batch has been aborted.

Total execution time: 00:00:00.002

In [15]:
select * FROM Arak

(30 rows affected)

Total execution time: 00:00:00.006

KocsmaID,ItalID,Ar
1,1,8
1,2,16
1,3,500
1,4,9
1,5,6
1,8,45
1,9,25
1,10,29
2,2,38
2,7,39


In [9]:
delete from Arak where Ar=1500

(1 row affected)

Total execution time: 00:00:00.014

5. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span> **DELETE triggert**<span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">, amely nem enged törölni olyan kocsmát, melyet legalább 3-an kedvelnek és a nyitás dátuma 2015 után történt!</span>

In [82]:
CREATE OR ALTER TRIGGER NemTorolKocsma ON Kocsmak
INSTEAD OF
DELETE
AS
BEGIN
    SET NOCOUNT ON
    IF EXISTS (
        SELECT k.KocsmaID FROM deleted d
        JOIN Kedvencek k ON d.KocsmaID = k.KocsmaID
        WHERE d.NyitasDatuma > '2015-01-01'
        GROUP BY k.KocsmaID
        HAVING COUNT(*) >= 3
    )
    BEGIN
        RAISERROR('Nem torolheto, 2015 utan nyitottak es legalabb 3-an kedvelik!', 16, 1)
        END
    ELSE
    BEGIN
        DELETE FROM Kocsmak
        WHERE KocsmaID IN (SELECT KocsmaID FROM deleted)
    END
END

Commands completed successfully.

Total execution time: 00:00:00.032

In [8]:
DELETE from Kocsmak where KocsmaID=2

: Msg 50000, Level 16, State 1, Procedure NemTorolKocsma, Line 15
Nem torolheto, 2015 utan nyitottak es legalabb 3-an kedvelik!

(1 row affected)

Total execution time: 00:00:00.164

**6.** **a.** <span style="color: var(--vscode-foreground);">&nbsp;Hozzunk létre egy új táblát:</span>

  Arak\_Log(ID, Idopont, Muvelet, KocsmaID, KocsmaNev, ItalID, ItalNev, RegiAr, UjAr),

ahol Idopont-datetime, Muvelet-varchar(50), RegiAr-int, UjAr-int típusú mezők.

In [5]:
CREATE TABLE Arak_Log(
    ID INT PRIMARY KEY, 
    Idopont DATETIME,
    Muvelet VARCHAR(50),
    KocsmaID INT FOREIGN KEY REFERENCES Kocsmak(KocsmaID),
    KocsmaNev VARCHAR(30),
    ItalID INT FOREIGN KEY REFERENCES Italok(ItalID),
    ItalNev VARCHAR(30),
    RegiAr INT,
    UjAr INT,
)

Commands completed successfully.

Total execution time: 00:00:00.017

6\. b. <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Oldjuk meg, hogy a&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Kocsmak táblából való törlés esetén az Arak_Log tábla&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">KocsmaID</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">mezőjének értéke&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">NULL-ra módosuljon!</span>

In [80]:
ALTER TABLE Arak_Log
DROP CONSTRAINT FK__Arak_Log__Kocsma__71D1E811

ALTER TABLE Arak_Log
ADD CONSTRAINT FK__Arak_Log__Kocsma__71D1E811
FOREIGN KEY (KocsmaID)
REFERENCES Kocsmak(KocsmaID)
ON DELETE SET NULL

Commands completed successfully.

Total execution time: 00:00:00.075

In [81]:
SELECT name
FROM sys.foreign_keys
WHERE parent_object_id = OBJECT_ID('Arak_Log')

(2 rows affected)

Total execution time: 00:00:00.016

name
FK__Arak_Log__ItalID__72C60C4A
FK__Arak_Log__Kocsma__71D1E811


6\. c. <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Írjunk&nbsp;</span> **UPDATE triggert**<span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">, mely az Arak tábla Ar mezőjének módosításakor beszúr egy új sort a Arak_Log táblába. Az időpont legyen a módosítás időpontja, művelet szövege: ‘ar növelése’ vagy ‘ar csökkentése’, RegiAr-régi érték, UjAr-új érték.</span>

In [84]:
CREATE TRIGGER Arak_Update
ON Arak
AFTER UPDATE
AS
BEGIN
    DECLARE @RegiAr INT, @UjAr INT, @Muvelet VARCHAR(50)

    IF UPDATE(Ar)
    BEGIN
        SELECT @RegiAr = d.Ar, @UjAr = i.Ar
        FROM inserted i
        JOIN deleted d ON i.ItalID = d.ItalID AND i.KocsmaID = d.KocsmaID

        IF @RegiAr < @UjAr
            SET @Muvelet = 'ar novelese'
        ELSE IF @RegiAr > @UjAr
            SET @Muvelet = 'ar csokkentese'
        ELSE
            SET @Muvelet = 'ar modositasa nelkul'

        INSERT INTO Arak_Log (Idopont, Muvelet, KocsmaID, KocsmaNev, ItalID, ItalNev, RegiAr, UjAr)
        SELECT GETDATE(), @Muvelet, i.KocsmaID, k.Nev AS KocsmaNev, i.ItalID, it.Nev AS ItalNev, @RegiAr, @UjAr
        FROM inserted i
        JOIN deleted d ON i.ItalID = d.ItalID AND i.KocsmaID = d.KocsmaID
        JOIN Kocsmak k ON i.KocsmaID = k.KocsmaID
        JOIN Italok it ON i.ItalID = it.ItalID
    END
END

Commands completed successfully.

Total execution time: 00:00:00.010